In [ ]:
import yfinance as yf
import pandas as pd

def get_sp500_tickers():
    df = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
    df['Symbol'] = df['Symbol'].str.replace('.', '-', regex=False)
    return df

def get_market_caps(tickers):
    market_caps = {}
    for ticker in tickers:
        try:
            info = yf.Ticker(ticker).info
            market_caps[ticker] = info.get("marketCap", 0)
        except Exception:
            market_caps[ticker] = 0
    return market_caps

def download_top100_data(tickers, start, end):
    print("⏳ Downloading top 100 tickers in batch...")
    data = yf.download(tickers, start=start, end=end, group_by='ticker', threads=True)
    combined = []
    for ticker in tickers:
        if ticker in data.columns.levels[0]:
            df = data[ticker].copy()
            df['Ticker'] = ticker
            df = df.reset_index()
            combined.append(df)
    return pd.concat(combined)


start = "2010-01-01"
end = "2025-04-15"

sp500_df = get_sp500_tickers()
tickers = sp500_df['Symbol'].tolist()
market_caps = get_market_caps(tickers)

top100_tickers = sorted(market_caps, key=market_caps.get, reverse=True)[:100]

top100_data = download_top100_data(top100_tickers, start, end)
top100_data.to_csv("top100.csv", index=False)

print("✅ Done! Saved top 100 S&P 500 data to top100.csv")


⏳ Downloading top 100 tickers in batch...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  100 of 100 completed


✅ Done! Saved top 100 S&P 500 data to top100.csv


In [3]:
data = pd.read_csv('top100.csv')

data.head()

,Date,Open,High,Low,Close,Volume,Ticker
0,2010-01-04,6.422876,6.455076,6.391278,6.440330,493729600.0,AAPL
1,2010-01-05,6.458086,6.487879,6.417459,6.451466,601904800.0,AAPL
2,2010-01-06,6.451465,6.477044,6.342225,6.348845,552160000.0,AAPL
3,2010-01-07,6.372319,6.379843,6.291066,6.337109,477131200.0,AAPL
4,2010-01-08,6.328684,6.379844,6.291369,6.379241,447610800.0,AAPL


In [4]:
data.shape


(384400, 7)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384400 entries, 0 to 384399
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Date    384400 non-null  object 
 1   Open    374090 non-null  float64
 2   High    374090 non-null  float64
 3   Low     374090 non-null  float64
 4   Close   374090 non-null  float64
 5   Volume  374090 non-null  float64
 6   Ticker  384400 non-null  object 
dtypes: float64(5), object(2)
memory usage: 20.5+ MB


In [6]:
data.isnull().sum()

Date          0
Open      10310
High      10310
Low       10310
Close     10310
Volume    10310
Ticker        0
dtype: int64

In [ ]:
import pandas as pd


data = pd.read_csv("top100.csv", parse_dates=["Date"])
possible_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
ohlcv_cols = [col for col in possible_cols if col in data.columns]
data = data.dropna(subset=ohlcv_cols, how='all')
data = data.sort_values(by=["Ticker", "Date"])

for col in ohlcv_cols:
    data[col] = data.groupby("Ticker")[col].transform(lambda x: x.ffill().bfill())

# Drop any rows that still contain missing values
data = data.dropna()

# Save cleaned version
data.to_csv("top100.csv", index=False)
print("✅ Cleaned data saved to top100.csv")


✅ Cleaned data saved to top100.csv


In [8]:
data['Date'] = pd.to_datetime(data['Date'], format='mixed', dayfirst=True, errors='coerce')

In [9]:
data.isnull().sum()

Date      0
Open      0
High      0
Low       0
Close     0
Volume    0
Ticker    0
dtype: int64

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 374090 entries, 0 to 65347
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    374090 non-null  datetime64[ns]
 1   Open    374090 non-null  float64       
 2   High    374090 non-null  float64       
 3   Low     374090 non-null  float64       
 4   Close   374090 non-null  float64       
 5   Volume  374090 non-null  float64       
 6   Ticker  374090 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 22.8+ MB


In [ ]:
import pandas as pd

# Define season mapping
season_mapper = {
    "WINTER": ["January", "February", "March"],
    "SPRING": ["April", "May", "June"],
    "SUMMER": ["July", "August", "September"],
    "FALL": ["October", "November", "December"]
}

def get_season(date):
    month = date.strftime('%B')
    for season, months in season_mapper.items():
        if month in months:
            return season
    return "UNKNOWN"


df = pd.read_csv("top100.csv")
df['Date'] = pd.to_datetime(df['Date'], format='mixed', dayfirst=True, errors='coerce')
df = df.dropna(subset=['Date'])
df['Season'] = df['Date'].apply(get_season)
seasonal_agg = df.groupby(['Ticker', 'Season'])['Close'].mean().unstack(fill_value=0)
for season in ["WINTER", "SPRING", "SUMMER", "FALL"]:
    if season not in seasonal_agg.columns:
        seasonal_agg[season] = 0


seasonal_agg = seasonal_agg[['WINTER', 'SPRING', 'SUMMER', 'FALL']]
seasonal_agg = seasonal_agg.reset_index()
seasonal_agg['Highest Avg Season'] = seasonal_agg[['WINTER', 'SPRING', 'SUMMER', 'FALL']].idxmax(axis=1)
seasonal_agg.rename(columns={
    'WINTER': 'Avg Close Winter',
    'SPRING': 'Avg Close Spring',
    'SUMMER': 'Avg Close Summer',
    'FALL': 'Avg Close Fall'
}, inplace=True)

seasonal_agg.to_csv("seasonal_stock_analysis_top100.csv", index=False)
print(seasonal_agg.head())


Season Ticker  Avg Close Winter  Avg Close Spring  Avg Close Summer  \
0        AAPL         71.442780         64.586175         71.439470   
1        ABBV         84.765713         78.168356         79.450597   
2         ABT         61.441884         57.951817         59.722630   
3         ACN        157.873994        146.367895        154.740318   
4        ADBE        224.114561        215.021057        242.155011   

Season  Avg Close Fall Highest Avg Season  
0            74.210849               FALL  
1            82.718044             WINTER  
2            61.074767             WINTER  
3           162.789591               FALL  
4           241.698435             SUMMER  


In [ ]:
import pandas as pd
import plotly.express as px

seasonal_agg = pd.read_csv("seasonal_stock_analysis_top100.csv")
seasonal_melted = seasonal_agg.melt(id_vars=['Ticker', 'Highest Avg Season'], 
                                    var_name='Season', value_name='Avg Close')
fig = px.bar(seasonal_melted, 
             x='Season', 
             y='Avg Close', 
             color='Season',
             title="Seasonal Stock Close Averages",
             labels={'Avg Close': 'Average Close Price'},
             animation_frame="Ticker")  # Creates an interactive dropdown for tickers

fig.update_layout(xaxis_title="Season", yaxis_title="Average Close Price", xaxis={'categoryorder':'array', 'categoryarray':['WINTER', 'SPRING', 'SUMMER', 'MONSOON']})

fig.show()


In [ ]:
import pandas as pd
import ta
data = pd.read_csv("top100.csv", parse_dates=["Date"])

def add_targets(data):
    data = data.sort_values(by=['Ticker', 'Date']).copy()
    data['target_1d'] = data.groupby('Ticker')['Close'].shift(-1)
    data['target_5d'] = data.groupby('Ticker')['Close'].shift(-5)
    data['target_20d'] = data.groupby('Ticker')['Close'].shift(-20)

    # Directional targets: 1 = price goes up, 0 = price goes down or same
    data['direction_1d'] = (data['target_1d'] > data['Close']).astype(int)
    data['direction_5d'] = (data['target_5d'] > data['Close']).astype(int)
    data['direction_20d'] = (data['target_20d'] > data['Close']).astype(int)

    return data

data_with_targets = add_targets(data)
data_with_targets = data_with_targets.dropna().reset_index(drop=True)
data_with_targets.to_csv("top100_targets.csv", index=False)
print("✅ Added 1D, 5D, 20D targets — saved as top100_targets.csv")



def add_features(data, lag_days=5):
    data = data.sort_values(by=["Ticker", "Date"]).copy()
    for lag in range(1, lag_days + 1):
        data[f'Close_lag_{lag}'] = data.groupby('Ticker')['Close'].shift(lag)
        data[f'Volume_lag_{lag}'] = data.groupby('Ticker')['Volume'].shift(lag)
   
   
    data['rsi'] = data.groupby('Ticker')['Close'].transform(lambda x: ta.momentum.RSIIndicator(x).rsi())
    data['ema_12'] = data.groupby('Ticker')['Close'].transform(lambda x: ta.trend.EMAIndicator(x, window=12).ema_indicator())
    data['ema_26'] = data.groupby('Ticker')['Close'].transform(lambda x: ta.trend.EMAIndicator(x, window=26).ema_indicator())

    macd = data.groupby('Ticker')['Close'].transform(lambda x: ta.trend.MACD(x).macd_diff())
    data['macd_diff'] = macd
    data['bb_high'] = data.groupby('Ticker')['Close'].transform(lambda x: ta.volatility.BollingerBands(x).bollinger_hband())
    data['bb_low'] = data.groupby('Ticker')['Close'].transform(lambda x: ta.volatility.BollingerBands(x).bollinger_lband())
    data['sma_20'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).mean())
    data['std_20'] = data.groupby('Ticker')['Close'].transform(lambda x: x.rolling(window=20).std())
    data = data.dropna().reset_index(drop=True)
    return data

data_with_features = add_features(data_with_targets)
data_with_features.to_csv("top100_features.csv", index=False)
print("✅ Features added and saved to top100_features.csv")


✅ Added 1D, 5D, 20D targets — saved as top100_targets.csv
✅ Features added and saved to top100_features.csv


In [ ]:
import pandas as pd
data = pd.read_csv("top100_targets.csv")
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by=['Ticker', 'Date']).reset_index(drop=True)
data.to_csv("top100_targets.csv", index=False)

print("✅ 'Date' column converted to datetime and file updated!")


✅ 'Date' column converted to datetime and file updated!


In [15]:
data = pd.read_csv("top100_targets.csv")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 372090 entries, 0 to 372089
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           372090 non-null  object 
 1   Open           372090 non-null  float64
 2   High           372090 non-null  float64
 3   Low            372090 non-null  float64
 4   Close          372090 non-null  float64
 5   Volume         372090 non-null  float64
 6   Ticker         372090 non-null  object 
 7   target_1d      372090 non-null  float64
 8   target_5d      372090 non-null  float64
 9   target_20d     372090 non-null  float64
 10  direction_1d   372090 non-null  int64  
 11  direction_5d   372090 non-null  int64  
 12  direction_20d  372090 non-null  int64  
dtypes: float64(8), int64(3), object(2)
memory usage: 36.9+ MB


In [16]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from statsmodels.tsa.arima.model import ARIMA


In [17]:
# 2.1 Directional Accuracy
def directional_accuracy(actual, predicted):
    actual_direction = (np.diff(actual) > 0).astype(int)
    predicted_direction = (np.diff(predicted) > 0).astype(int)
    return np.mean(actual_direction == predicted_direction) * 100


In [18]:
# from sklearn.metrics import precision_score, recall_score

# def precision_recall(actual, predicted):
#     actual_direction = (np.diff(actual) > 0).astype(int)
#     predicted_direction = (np.diff(predicted) > 0).astype(int)

#     precision = precision_score(actual_direction, predicted_direction, zero_division=0)
#     recall = recall_score(actual_direction, predicted_direction, zero_division=0)
#     return precision * 100, recall * 100


In [19]:
def save_text_outputs(ticker, current_price, date, results):
    import os
    os.makedirs("stock_prediction_results", exist_ok=True)

    metrics_path = f"stock_prediction_results/{ticker}_metrics.txt"
    prediction_path = f"stock_prediction_results/{ticker}_prediction.txt"

    # --- UNIFIED METRICS ---
    with open(metrics_path, "w") as f:
        f.write(f"Combined Performance Metrics (Averaged across ARIMA, XGBoost, LSTM)\n\n")
        for horizon in ["1", "5", "20"]:
            try:
                rmse = np.mean([results[model]["metrics"][horizon]["rmse"] for model in results])
                mae = np.mean([results[model]["metrics"][horizon]["mae"] for model in results])
                acc = np.mean([results[model]["metrics"][horizon]["directional_acc"] for model in results])

                f.write(f"{horizon}-Day Horizon:\n")
                f.write(f"- Avg RMSE: {rmse:.2f}\n")
                f.write(f"- Avg MAE : {mae:.2f}\n")
                f.write(f"- Avg Directional Accuracy: {acc:.1f}%\n\n")
                


            except Exception as e:
                f.write(f"{horizon}-Day Horizon: Error collecting metrics\n\n")

    # --- UNIFIED PREDICTIONS ---
    with open(prediction_path, "w") as f:
        f.write(f"Stock: {ticker} | Date: {date.date()} | Current Price: ${current_price:.2f}\n\n")
        f.write("Combined Multi-Horizon Predictions (Ensembled):\n")

        for horizon in ["1", "5", "20"]:
            try:
                # Gather predictions
                preds = [results[model]["predictions"][horizon]["value"] for model in results]
                stds = [results[model]["predictions"][horizon]["std"] for model in results]

                ensemble_pred = np.mean(preds)
                lower = ensemble_pred - np.max(stds)
                upper = ensemble_pred + np.max(stds)
                pred_date = pd.to_datetime(date) + pd.Timedelta(days=int(horizon))

                f.write(f"- {horizon}-day ({pred_date.date()}): ${ensemble_pred:.2f} (95% CI: ${lower:.2f} – ${upper:.2f})\n")
            except Exception as e:
                f.write(f"- {horizon}-day: Prediction error\n")

        f.write("\nDirection Forecast (Majority Vote):\n")
        for horizon in ["1", "5", "20"]:
            try:
                directions = [results[model]["predictions"][horizon]["direction"] for model in results]
                ups = directions.count("UP")
                downs = directions.count("DOWN")
                final_direction = "UP" if ups >= 2 else "DOWN"
                confidence = int((max(ups, downs) / 3) * 100)
                f.write(f"- {horizon}-day: {final_direction} ({confidence}% agreement)\n")
                if confidence >= 80:
                    f.write(f"  🚨 High-confidence {final_direction} prediction!\n")
            except:
                f.write(f"- {horizon}-day: N/A\n")

    print(f"✅ Saved unified metrics to {metrics_path}")
    print(f"✅ Saved unified predictions to {prediction_path}")


In [20]:
# ARIMA 


from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_arima(ticker, data_path="top100.csv"):
    data = pd.read_csv(data_path, parse_dates=["Date"])
    stock_data = data[data["Ticker"] == ticker].sort_values("Date")

    close = stock_data["Close"].values
    dates = stock_data["Date"].values
    train_size = int(len(close) * 0.7)
    train, test = close[:train_size], close[train_size:]
    test_dates = dates[train_size:]

    result = {
        "metrics": {},
        "predictions": {}
    }

    for horizon in [1, 5, 20]:
        try:
            model = ARIMA(train, order=(5, 1, 0))
            fit = model.fit()
            forecast = fit.forecast(steps=len(test))

            actual = test[horizon:]
            preds = forecast[:-horizon]
            rmse = np.sqrt(mean_squared_error(actual, preds))
            mae = mean_absolute_error(actual, preds)
            acc = directional_accuracy(actual, preds)

            result["metrics"][str(horizon)] = {
                "rmse": rmse,
                "mae": mae,
                "directional_acc": acc
            }

            pred_price = preds[-1]
            std = np.std(preds)
            direction = "UP" if pred_price > close[-1] else "DOWN"

            result["predictions"][str(horizon)] = {
                "value": pred_price,
                "std": std,
                "direction": direction
            }

        except Exception as e:
            print(f"❌ ARIMA failed for {ticker} ({horizon}-day): {e}")
    return result


In [21]:
# XGBoost 


import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

def run_xgboost(ticker, data_path="top100_features.csv"):
    data = pd.read_csv(data_path, parse_dates=["Date"])
    stock_data = data[data["Ticker"] == ticker].sort_values("Date")

    result = {
        "metrics": {},
        "predictions": {}
    }

    for horizon in ["1d", "5d", "20d"]:
        target_col = f"target_{horizon}"
        if target_col not in stock_data.columns:
            continue

        # Drop other targets/directions
        drop_cols = [col for col in stock_data.columns if "target_" in col or "direction_" in col]
        drop_cols.remove(target_col)
        model_data = stock_data.drop(columns=drop_cols + ["Ticker"])

        X = model_data.drop(columns=["Date", target_col])
        y = model_data[target_col]
        dates = model_data["Date"]

        split = int(len(X) * 0.7)
        X_train, X_test = X.iloc[:split], X.iloc[split:]
        y_train, y_test = y.iloc[:split], y.iloc[split:]
        date_test = dates.iloc[split:]

        model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=5)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, preds))
        mae = mean_absolute_error(y_test, preds)
        acc = directional_accuracy(y_test.values, preds)


        pred_price = preds[-1]
        std = np.std(preds)
        direction = "UP" if pred_price > stock_data["Close"].iloc[-1] else "DOWN"

        result["metrics"][horizon.replace("d", "")] = {
            "rmse": rmse,
            "mae": mae,
            "directional_acc": acc
        }

        result["predictions"][horizon.replace("d", "")] = {
            "value": pred_price,
            "std": std,
            "direction": direction
        }
        
    return result


In [ ]:
#LSTM

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

def create_sequences(data, window_size, horizon):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size+horizon-1])  # horizon-ahead target
    return np.array(X), np.array(y)

def run_lstm(ticker, data_path="top100.csv", window_size=20):
    data = pd.read_csv(data_path, parse_dates=["Date"])
    stock_data = data[data["Ticker"] == ticker].sort_values("Date")
    close_prices = stock_data["Close"].values.reshape(-1, 1)
    dates = stock_data["Date"].values

    # Scale prices
    scaler = MinMaxScaler()
    scaled_prices = scaler.fit_transform(close_prices)

    result = {
        "metrics": {},
        "predictions": {}
    }

    for horizon in [1, 5, 20]:
        try:
            X, y = create_sequences(scaled_prices, window_size, horizon)
            split = int(len(X) * 0.7)
            X_train, X_test = X[:split], X[split:]
            y_train, y_test = y[:split], y[split:]

            # Reshape for LSTM: (samples, timesteps, features)
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

            # Model
            model = Sequential([
                LSTM(50, activation='relu', input_shape=(window_size, 1)),
                Dense(1)
            ])
            model.compile(optimizer='adam', loss='mse')
            model.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)

            preds_scaled = model.predict(X_test)
            y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
            preds_inv = scaler.inverse_transform(preds_scaled).flatten()

            rmse = np.sqrt(mean_squared_error(y_test_inv, preds_inv))
            mae = mean_absolute_error(y_test_inv, preds_inv)
            acc = directional_accuracy(y_test_inv, preds_inv)

            pred_price = preds_inv[-1]
            std = np.std(preds_inv)
            direction = "UP" if pred_price > close_prices[-1] else "DOWN"

            result["metrics"][str(horizon)] = {
                "rmse": rmse,
                "mae": mae,
                "directional_acc": acc
            }
            result["predictions"][str(horizon)] = {
                "value": pred_price,
                "std": std,
                "direction": direction
            }

            print(f"✅ LSTM | {ticker} | {horizon}-day ➜ RMSE: {rmse:.2f}, MAE: {mae:.2f}")

        except Exception as e:
            print(f"❌ LSTM failed for {ticker} ({horizon}-day): {e}")
        
    return result


In [ ]:
def run_full_prediction(ticker):
    print(f"\n🚀 Running full prediction system for {ticker}...\n")
    data_clean = pd.read_csv("top100.csv", parse_dates=["Date"])
    stock_data = data_clean[data_clean["Ticker"] == ticker].sort_values("Date")
    
    stock_data = data_clean[data_clean["Ticker"] == ticker]

    if stock_data.empty:
        print(f"❌ No data found for {ticker} in top100.csv")
        return

    stock_data = stock_data.sort_values("Date")

    latest_row = stock_data.iloc[-1]

    
    latest_row = stock_data.iloc[-1]
    current_price = latest_row["Close"]
    latest_date = latest_row["Date"]
    xgb_results = run_xgboost(ticker)
    arima_results = run_arima(ticker)
    lstm_results = run_lstm(ticker)

    results = {
        "XGBoost": xgb_results,
        "ARIMA": arima_results,
        "LSTM": lstm_results
    }
    save_text_outputs(ticker, current_price, latest_date, results)


if __name__ == "__main__":
    import pandas as pd

    print("📊 Stock Market Multi-Horizon Prediction System (ARIMA + XGBoost + LSTM)")
    print("--------------------------------------------------------------")

    ticker = input("🔎 Enter stock ticker (e.g., AAPL, GOOGL, BKNG): ").upper()

    # Optional: Check if ticker is in the dataset
    try:
        data = pd.read_csv("top100.csv")
        if ticker not in data["Ticker"].unique():
            print(f"❌ Ticker '{ticker}' not found in dataset.")
        else:
            run_full_prediction(ticker)
    except Exception as e:
        print(f"❌ Failed to load data or run model: {e}")



📊 Stock Market Multi-Horizon Prediction System (ARIMA + XGBoost + LSTM)
--------------------------------------------------------------

🚀 Running full prediction system for ABBV...



/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
✅ LSTM | ABBV | 1-day ➜ RMSE: 4.49, MAE: 3.57


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
✅ LSTM | ABBV | 5-day ➜ RMSE: 6.23, MAE: 4.54


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
✅ LSTM | ABBV | 20-day ➜ RMSE: 21.01, MAE: 15.07
✅ Saved unified metrics to stock_prediction_results/ABBV_metrics.txt
✅ Saved unified predictions to stock_prediction_results/ABBV_prediction.txt
